In [ ]:
!pip install transformers
!pip install sentencepiece
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.4 MB 24.5 MB/s 
     |████████████████████████████████| 101 kB 9.3 MB/s 
     |████████████████████████████████| 596 kB 65.9 MB/s 
     |████████████████████████████████| 6.6 MB 59.0 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.2 MB 30.5 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 362 kB 2.4 MB/s 
     |████████████████████████████████| 212 kB 14.2 MB/s 
     |████████████████████████████████| 140 kB 74.1 MB/s 
     |████████████████████████████████| 1.1 MB 46.2 MB/s 
     |████████████████████████████

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from tqdm import tqdm
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [ ]:
#df = pd.read_csv('/content/drive/MyDrive/clean_df.csv')

In [ ]:
#df = df[:200000]

In [ ]:
#df.to_csv('reduced_gec.csv')

In [ ]:
#!cp reduced_gec.csv /content/drive/MyDrive

In [ ]:
from transformers import BartTokenizerFast, TFBartForConditionalGeneration, DataCollatorForSeq2Seq,TFAutoModelForSeq2SeqLM,AutoTokenizer,AutoConfig,KerasMetricCallback
import tensorflow as tf


t5_small = "t5-small"
trf_tokenizer = t5_small
trf_model = t5_small
tokenizer = AutoTokenizer.from_pretrained(trf_tokenizer)


Downloading:   0%|          | 0.00/1.17k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/773k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.32M [00:00<?, ?B/s]

/usr/local/lib/python3.7/dist-packages/transformers/models/t5/tokenization_t5_fast.py:166: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-small automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  FutureWarning,


In [ ]:
from datasets import load_dataset
raw_datasets = load_dataset('csv', data_files='/content/drive/MyDrive/reduced_gec.csv')

Using custom data configuration default-89e505a623b7542d


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

0 tables [00:00, ? tables/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-89e505a623b7542d/0.0.0/51cce309a08df9c4d82ffd9363bbe090bf173197fc01a71b034e8594995a1a58. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
def preprocess_tokenizer(data):
  inputs = ['gec: '+i for i in data["Correct"]]
  targets = [j for j in data["Incorrect"]]
  model_inputs = tokenizer(inputs,truncation=True)

  with tokenizer.as_target_tokenizer():
      labels = tokenizer(targets,truncation=True)

  model_inputs["labels"] = labels["input_ids"]
  return model_inputs

In [ ]:
raw_dataset = raw_datasets["train"].train_test_split(test_size=0.2)

In [ ]:
tokenized_sentences = raw_dataset.map(preprocess_tokenizer, batched=True)

Parameter 'function'=<function preprocess_tokenizer at 0x7f4288dc2b90> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


  0%|          | 0/160 [00:00<?, ?ba/s]

  0%|          | 0/40 [00:00<?, ?ba/s]

In [ ]:
config = AutoConfig.from_pretrained(trf_model)
model = TFAutoModelForSeq2SeqLM.from_pretrained(trf_model, config=config)

Downloading:   0%|          | 0.00/231M [00:00<?, ?B/s]

All model checkpoint layers were used when initializing TFT5ForConditionalGeneration.

All the layers of TFT5ForConditionalGeneration were initialized from the model checkpoint at t5-small.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.


In [ ]:
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model, return_tensors="tf")

In [ ]:
tf_train_set = tokenized_sentences["train"].to_tf_dataset(
    columns=["attention_mask", "input_ids", "labels"],
    shuffle=True,
    batch_size=16,
    collate_fn=data_collator,
)

tf_test_set = tokenized_sentences["test"].to_tf_dataset(
    columns=["attention_mask", "input_ids", "labels"],
    shuffle=False,
    batch_size=16,
    collate_fn=data_collator,
)

In [ ]:
from transformers import create_optimizer, AdamWeightDecay

optimizer = AdamWeightDecay(learning_rate=2e-5, weight_decay_rate=0.01)

In [ ]:
model.compile(optimizer=optimizer,loss='sparse_categorical_crossentropy')

In [ ]:
model.fit(x=tf_train_set, validation_data=tf_test_set, epochs=1)
model.save_pretrained("/content/drive/MyDrive/t5")

10000/10000 [==============================] - 1356s 136ms/step - loss: nan - val_loss: nan


In [ ]:
inputs = tokenizer("gec: how is", truncation=True, return_tensors="tf")
output = model.generate(**inputs, num_beams=8, do_sample=True)[0][1:]
decoded_output = tokenizer.batch_decode(output)

In [ ]:
decoded_output